In [ ]:
!pip3 install "paddleocr>=2.6.0.3"
!pip install paddlepaddle==3.0.0b1
!pip3 install paddleclas>=2.4.3

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

In [ ]:
!pip install premailer

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install pillow_avif-plugin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 58.8 MB/s eta 0:00:00


In [ ]:
!sudo apt get tesseract-ocr
!pip install pytesseract
!pip install opencv-python

In [ ]:
from paddleocr import PaddleOCR,draw_ocr
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory

In [ ]:
from os.path import dirname
import os
from PIL import Image
import pillow_avif
def img_pathwfc(dirname, files):
  files = 'DR9rZFEUQAAu5l5.jpeg'
  dirname = '/content'
  if files.endswith('.avif'):
    filename = files.removesuffix('.avif')
    img = Image.open(os.path.join(dirname, files))
    os.remove(os.path.join(dirname, files))
    img.save(os.path.join(dirname, f'{filename}.jpg'))
    img_path = os.path.join(dirname, f'{filename}.jpg')
  elif files.endswith('.jpg_large'):
    os.rename(os.path.join(dirname, files), os.path.join(dirname, files.replace('_large', '')))
    img_path = os.path.join(dirname, files.replace('_large', ''))
  else:
    img_path = os.path.join(dirname, files)
  return img_path

In [ ]:
def ocr(img_path):
  result = ocr.ocr(img_path, cls=True)
  coords={}
  heights_sum = 0
  n = 0
  img = cv2.imread(img_path)
  custom_config = r'--oem 3 --psm 6'
  out_str = pytesseract.image_to_string(img, config=custom_config)
  l = len(out_str)
  paddle_run = True
  if l > 500:
    paddle_run = False
  if paddle_run:
    for idx in range(len(result)):
        res = result[idx]
        for line in res:
          x = (line[0][0][0] + line[0][1][0] + line[0][2][0] + line[0][3][0])/4
          y = (line[0][0][1] + line[0][1][1] + line[0][2][1] + line[0][3][1])/4
          heights_sum += abs(line[0][2][1] - line[0][0][1])
          n += 1
          coords[y] = [x, line[1][0]]
          print(line)
    height_avg = heights_sum / n
    print(height_avg)

    # Sort coordinates by keys
    myKeys = list(coords.keys())
    myKeys.sort()
    sorted_coords = {i: coords[i] for i in myKeys}
    print(sorted_coords)

    out_str = ''
    i = 0
    while i < len(myKeys):
        j = i
        # Collect words while the difference between keys is less than height_avg
        while j < len(myKeys) and abs(myKeys[j] - myKeys[i]) < height_avg:
            out_str += sorted_coords[myKeys[j]][1] + ' '
            j += 1
        out_str += '\n'
        # Move to the next segment
        i = j
  return out_str



[2024/09/05 17:10:54] ppocr DEBUG: dt_boxes num : 151, elapsed : 0.9634332656860352
[2024/09/05 17:10:54] ppocr DEBUG: cls num  : 151, elapsed : 0.607696533203125
[2024/09/05 17:11:51] ppocr DEBUG: rec_res num  : 151, elapsed : 57.227357149124146
[[[586.0, 56.0], [875.0, 33.0], [878.0, 69.0], [589.0, 93.0]], ('METRQ SrATION Phuce 1', 0.6514522433280945)]
[[[636.0, 88.0], [809.0, 74.0], [813.0, 110.0], [639.0, 125.0]], ('DELHI-0C9', 0.5609809160232544)]
[[[579.0, 129.0], [886.0, 104.0], [889.0, 140.0], [582.0, 165.0]], ('TELNO.01-85001-957', 0.6699370741844177)]
[[[579.0, 165.0], [880.0, 142.0], [882.0, 176.0], [582.0, 199.0]], ('HELFLINE18002002255', 0.8293232321739197)]
[[[470.0, 208.0], [995.0, 168.0], [998.0, 204.0], [473.0, 244.0]], ('GST TINO7AACCS1Q93NIEF QQT.2', 0.7189303636550903)]
[[[555.0, 240.0], [917.0, 212.0], [919.0, 249.0], [558.0, 276.0]], ('CIN NOL5790SN2007FLC2E8269', 0.6984544992446899)]
[[[924.0, 282.0], [1024.0, 277.0], [1026.0, 315.0], [927.0, 321.0]], ('NET A', 0

32.81884057971015
{62.75: [732.0, 'METRQ SrATION Phuce 1'], 99.25: [724.25, 'DELHI-0C9'], 134.5: [734.0, 'TELNO.01-85001-957'], 170.5: [730.75, 'HELFLINE18002002255'], 206.0: [734.0, 'GST TINO7AACCS1Q93NIEF QQT.2'], 244.25: [737.25, 'CIN NOL5790SN2007FLC2E8269'], 298.75: [975.25, 'NET A'], 309.25: [810.5, 'QTY QISC A'], 328.5: [482.0, 'ITEM DESC'], 346.75: [811.75, 'CGSTXSSSTX'], 363.0: [555.0, 'UOM'], 368.0: [448.0, 'HSN'], 441.5: [994.0, '79.00'], 449.0: [872.5, 'Q.00'], 453.5: [766.0, '1'], 462.0: [535.5, 'FLUSH TABLT 2*50G'], 485.0: [854.0, '9'], 488.5: [765.0, '6'], 498.0: [557.5, 'Pcs'], 500.0: [458.0, '3402'], 514.0: [991.5, '138.00'], 519.0: [874.0, '14.00-'], 522.5: [766.0, '1'], 530.0: [554.25, 'HARPIC POW PLS OG 11'], 555.0: [857.5, '9'], 559.0: [767.0, '9'], 566.0: [557.5, 'Pcs'], 568.5: [460.5, '3808'], 581.25: [992.75, '200.00'], 588.0: [876.0, '0.00'], 591.0: [768.0, '1'], 597.5: [554.0, 'COMFRT LILY FR 800m'], 624.0: [860.0, '9'], 626.0: [769.0, '9'], 631.5: [562.0, 'Pc

In [ ]:
from dotenv import load_dotenv
import google.generativeai as genai
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key (Ensure it's valid and has text generation permission)
from google.colab import userdata

def extract_invoice_details(invoice_text):
    """
    Extract customer details, products, and total amount from an unstructured invoice text.

    Parameters:
    - invoice_text: The unstructured text data of an invoice.

    Returns:
    - A structured response with customer details, products, and total amount.
    """
    try:
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")

        # Construct a prompt to extract customer details, products, and total amount
        prompt = (
            "Extract the following details from this receipt text:"
            'Fix the spelling mistakes '
            "1. Merchant Name."
            "2. Invoice No.(Bill No./Receipt No./GST No. )"
            "3. Products (Name, Price)."
            "4. Total Amount."
            '5. Date( in DD/MM/YYYY)'
            "Provide the details in a structured format(json)."
        )

        response = model.generate_content([invoice_text, prompt])

        return response.text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#Usage:



Extracted Details:
```json
{
  "Merchant Name": "METRO",
  "Invoice No.": "7AACCS1Q93NIEF/QT.2",
  "Products": [
    {
      "Name": "FLUSH TABLET 2*50G",
      "Price": "79.00"
    },
    {
      "Name": "HARPIC POW PLS 1KG",
      "Price": "138.00"
    },
    {
      "Name": "COMFRT LILY FR 800ml",
      "Price": "200.00"
    },
    {
      "Name": "SCRUBBER 2 IN 1BIG",
      "Price": "120.00"
    },
    {
      "Name": "5.5 Sqr Noodle Bowl",
      "Price": "139.00"
    },
    {
      "Name": "5.5 Sqr Noodle Bowl",
      "Price": "139.00"
    },
    {
      "Name": "FCS",
      "Price": "88.00"
    },
    {
      "Name": "HMLY HROIL AHF 200ml",
      "Price": "84.00"
    },
    {
      "Name": "AMUL BUTTER TUB 200g",
      "Price": "162.00"
    },
    {
      "Name": "BRU GOLD JAR 50g",
      "Price": "94.00"
    },
    {
      "Name": "SURF EX DET BAR 95g",
      "Price": "144.00"
    },
    {
      "Name": "ARIEL TOP LOAD 4K+2K",
      "Price": "898.00"
    }
  ],
  "Total Amount":

In [ ]:
i = 0
dirname = '/content/dataset'
for files in os.listdir(dirname):
  GOOGLE_API_KEY=userdata.get(f'{i+1}')
  genai.configure(api_key=GOOGLE_API_KEY)
  img_path = img_pathwfc(dirname, files)
  out_str = ocr(img_path)
  response = extract_invoice_details(out_str)

  if response:
      print(response)
  else:
      print("No response generated.")
  i+=1
  if i ==7:
    i= 0